This code will generate the feature vector, using sub grids to calculate the overlap

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import numpy.matlib
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

##  Step 0. Throw out fires that have 0 viirs detects on any day. (fires are already filtered to be in the right lat/lon range

In [2]:
# load in the polygons and attributes
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
fire_attributes = pd.read_csv('fire_polygons_attributes.csv')
incidents = pd.read_csv('unique_fires.csv')
print(len(fire_polygons), len(fire_attributes), len(incidents))

#throw out fires that have 0 days with detects
daily_drop_inds = []
incident_drop_inds = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    if len(pts[pts==0])==len(pts):
        #print(item, 'oops all zeros')
        daily_drop_inds = np.append(daily_drop_inds, np.where(fire_attributes['Incident Number']==item)[0])
        incident_drop_inds = np.append(incident_drop_inds, np.where(incidents['Incident Number']==item)[0])

daily_drop_inds = [int(p) for p in daily_drop_inds]
#print(daily_drop_inds)

fire_attributes = fire_attributes.drop(labels= daily_drop_inds)
fire_attributes = fire_attributes.drop(labels='Unnamed: 0', axis=1)
fire_polygons = fire_polygons.drop(labels = daily_drop_inds)
incidents = incidents.drop(labels = incident_drop_inds)
print(len(fire_polygons), len(fire_attributes), len(incidents))

fire_attributes

22280 22280 1354
19137 19137 1036


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points
5,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0
6,10662684.0,PAINTED WAGON,2019-04-09,33.786944,-112.753333,0.0
7,10662684.0,PAINTED WAGON,2019-04-10,33.786944,-112.753333,0.0
8,10663171.0,LONE MOUNTAIN,2019-05-26,33.808056,-105.738611,0.0
9,10663171.0,LONE MOUNTAIN,2019-05-27,33.808056,-105.738611,0.0
...,...,...,...,...,...,...
22275,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0
22276,11980587.0,CREEK 5,2020-12-26,33.402646,-117.266795,0.0
22277,11980587.0,CREEK 5,2020-12-27,33.402646,-117.266795,0.0
22278,11980587.0,CREEK 5,2020-12-28,33.402646,-117.266795,0.0


## 1. Figure out the start and end date we will use for each fire

In [ ]:
# first check what fraction of the fire lifetimes are 0 days
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    #print(len(np.where(pts==0)[0]))
    print('The fire lifetime is ', len(np.where(pts!=0)[0])*100/len(pts), '% nonzero detections')

In [3]:
keep_days = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    dys = fire_attributes['Current Day'].values[fire_attributes['Incident Number']==item]
    nonzero = np.where(pts!=0)[0]
    first_day = dys[nonzero[0]]
    last_day = dys[nonzero[len(nonzero)-1]]
    keep_days = np.append(keep_days, np.where((fire_attributes['Incident Number']==item)&(fire_attributes['Current Day']>=first_day)&(fire_attributes['Current Day']<=last_day))[0])

keep_days = [int(p) for p in keep_days]
print(len(keep_days))
fire_attributes = fire_attributes.iloc[keep_days, :]
fire_polygons = fire_polygons.iloc[keep_days, :]
fire_attributes = fire_attributes.reset_index(drop=True) #reset the indices beecause we dropped some
fire_polygons = fire_polygons.reset_index(drop=True) #reset the indices beecause we dropped some

fire_polygons

8161


,geometry
0,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...
8156,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## 2. Loop through the days and get the raw features (temp at different pressure levels, wind, pressure, precip, terrain info) and the labels (QFED PM2.5 from the next day)

In [6]:
# THIS WAY IS PRETTY SLOW

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 
                          'Fire Name': np.nan*np.zeros(num_days), 
                         'Current Day': np.nan*np.zeros(num_days),
                        'Lat Fire': np.nan*np.zeros(num_days), 
                     'Lon Fire': np.nan*np.zeros(num_days), 
                         'Number of VIIRS points': np.nan*np.zeros(num_days),
                        'TLML': np.nan*np.zeros(num_days), 
                        'QLML': np.nan*np.zeros(num_days),
                                  'SPEEDLML': np.nan*np.zeros(num_days),
                                  'PS': np.nan*np.zeros(num_days),
            'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe

fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

varis = {'WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['TLML', 'QLML', 'SPEEDLML', 'PS']}
locs = {'WESTUS_MERRA2_400.inst1_2d_lfo_Nx.':['lat', 'lon']}
subfolders = {'/MERRA2/'}
file_suffix = '.nc4'

for ii in range(num_days): #loop over all the days
    #get the day
    year = fire_features['Current Day'][ii][0:4]
    month = fire_features['Current Day'][ii][5:7]
    day = fire_features['Current Day'][ii][8:10]
    #print(year, month, day)
    
    for folder in subfolders:
        path = '/data2/lthapa/'+str(year)+folder
        #print(path)
        
        #THIS COULD BE A SEPARATE HELPER FUNCTION
        for file_prefix in varis.keys(): # for each dataset
            path = path+file_prefix+year+month+day+file_suffix
            print(path)
            dataset = nc.Dataset(path)
            
            #get the lat and lon for the dataset
            loc_vals = []
            for loc in locs.values():
                loc_vals.append(loc)
            loc_vals = loc_vals[0]
            lat = dataset[loc_vals[0]][:]
            lon = dataset[loc_vals[1]][:]
            #turn it into a meshgrid
            LON, LAT = np.meshgrid(lon, lat)
            #get the full variable, put it to daily resolution
            for value in varis.values(): 
                for var in value:
                    variable = dataset[var][:]
                    variable = np.nanmean(variable, axis=0) #average across the day
                    #print(variable.shape, LON.shape, LAT.shape)
            
                    #find the intersection
                    overlaps = calculate_overlaps(LAT, LON, fire_features['geometry'][ii])
                    variable_weighted = np.multiply(variable,overlaps)
                    #save it to the dataframe
                    fire_features.loc[ii, var] = np.nansum(variable_weighted)
fire_features

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190408.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190609.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190610.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190611.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190612.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190613.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190614.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190615.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190616.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190617.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190618.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190619.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190620.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190721.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190715.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190525.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190713.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190714.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190715.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190716.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190717.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190718.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190719.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190720.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190721.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190722.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190723.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190804.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190805.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190806.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190807.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190808.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190809.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190810.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190811.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190812.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190827.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190828.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190829.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190830.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190831.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190901.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190902.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190903.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191028.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191024.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190819.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190821.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190822.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190823.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190824.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190825.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190826.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190827.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190828.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190829.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190830.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190831.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191003.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191004.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191005.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191006.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191007.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191008.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191009.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191010.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191011.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191012.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191013.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190705.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190706.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190707.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190708.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190709.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190710.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190711.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190712.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190713.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190624.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191007.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20191008.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190912.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190913.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190914.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190915.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190916.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190802.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190803.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190804.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190805.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190806.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190807.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190808.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190809.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190901.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190902.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190903.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190904.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190905.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190816.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190817.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190818.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190819.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190820.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190821.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.20190822.nc4
/data2/lthapa/2019/MERRA2/WESTUS_MERRA2_400.inst1_2

KeyboardInterrupt: 

In [6]:
# dictionaries that track the variables, dimensions, times and levels
varis = {'/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['biomass']}

#0=lat, 1=lon, 2=time, 3=level 
dims = {'/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['lat', 'lon', 'time']}

times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'biomass':12}
#days_forecast = 
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                             'biomass_12Z_today': np.nan*np.zeros(num_days),'biomass_12Z_tomorrow': np.nan*np.zeros(num_days),
                             'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe
#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
print(len(unique_days))
for dd in range(15):#len(unique_days)):

    date=unique_days[dd]
    yr = date[0:4]
    mo = date[5:7]
    dy = date[8:10]
    print(yr, mo,dy)
    #find the polygons for the given day

    for i in varis : #loop over the file names
        #get the path to the file
        path = '/data2/lthapa/'+yr+i
        if '*' in path:
            path = path.replace('*', yr+mo+dy)
        if np.datetime64(date)>=np.datetime64('2020-09-01') and np.datetime64(date)<=np.datetime64('2020-09-30'):
            path = path.replace('400', '401')
            
        #load the file in as a netcdf dataset
        print(path)
        dat = nc.Dataset(path)
        ndims = len(dims[i])
        lat = dat[dims[i][0]][:]
        lon = dat[dims[i][1]][:]
        t = dat[dims[i][2]][:]
        
        if len(lat.shape)==1:
            lon, lat = np.meshgrid(lon, lat)

        #calculate the areas of the lat/lon grid for the DATASET
        lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
        inds = np.where(fire_features['Current Day']==date)[0]
        polys = fire_features['geometry'].iloc[inds].values
        
        for p in range(len(inds)):
            tic = time.time()
            #add a halo to the polygon
            poly = polys[p]
            poly_buff = poly.buffer(1)
            # get the bounds of the buffered polygon
            w,s,e,n = poly_buff.bounds
            #print(w,s,e,n)
            s_ind = np.unique(np.argmin(np.abs(lat-s), axis=0))[0]
            n_ind = np.unique(np.argmin(np.abs(lat-n), axis=0))[0]
            w_ind = np.unique(np.argmin(np.abs(lon-w), axis=1))[0]
            e_ind = np.unique(np.argmin(np.abs(lon-e), axis=1))[0]
    
            
            #tic = time.time()
            overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners,s_ind,n_ind,w_ind,e_ind, poly) #calculate the overlap between the poly and the lat/lon grid
            print(np.where(overlaps!=0))
            #toc = time.time()
            #print('Time elapsed for calculating overlaps for polygon:', toc-tic)
            for j in varis[i]: #loop over the variable names, which are lists
                var = dat[j][:]
                #print('the dimensions of', j, 'are', var.shape, 'and', j, 'has', ndims, 'dimensions')

                if j in times.keys():
                    ind_time = np.argmin(np.abs(times[j]*60-t))
                    #filter the variable by time
                    var = var[ind_time, :, :]
                    #print(var.shape)
                    var_weighted = np.multiply(var,overlaps)
                    var_ave = np.nansum(var_weighted)
                    name_in_table = j+'_'+str(times[j])+'Z'
                    fire_features.loc[inds[p],name_in_table] = var_ave

            toc = time.time()
            print('Time to process polygon=',toc-tic)
        dat.close() 
#print(inds[p])
fire_features

442
2019 02 17
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190217.nc4
(array([18]), array([105]))
Time to process polygon= 0.02664971351623535
2019 02 18
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190218.nc4
(array([18]), array([105]))
Time to process polygon= 0.024091720581054688
2019 02 24
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190224.nc4
(array([96, 97]), array([205, 205]))
Time to process polygon= 0.025977373123168945
2019 03 06
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190306.nc4
(array([0, 3]), array([142, 142]))
Time to process polygon= 0.016811609268188477
2019 03 13
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190313.nc4
(array([47, 47]), array([233, 234]))
Time to process polygon= 0.02032613754272461
2019 03 15
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190315.nc4
(array([25]), array([89]))
Time to process polygon= 0.023951292037963867
2019 03 16
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190316.nc4
(arra

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,biomass_12Z_today,biomass_12Z_tomorrow,geometry,biomass_12Z
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,NaN,NaN,"POLYGON ((-112.75455 33.78632, -112.76012 33.7...",7.627477e-10
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8...",NaN
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8...",NaN
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8...",NaN
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8...",NaN
...,...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,NaN,NaN,"POLYGON ((-112.11669 34.11898, -112.12564 34.1...",NaN
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,NaN,NaN,"POLYGON ((-117.03702 33.90632, -117.04484 33.9...",NaN
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,NaN,NaN,"POLYGON ((-117.27595 33.37881, -117.27495 33.3...",NaN
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,NaN,NaN,"POLYGON ((-117.29549 33.36558, -117.29713 33.4...",NaN


In [72]:
fire_features
#therest = fire_features[['Incident Number', 'Fire Name', 'Current Day','Lat Fire', 'Lon Fire','Number of VIIRS points', 'TLML_12Z', 'QLML_12Z','SPEEDLML_12Z','PS_12Z','T_12Z_700mb','T_12Z_500mb','QV_12Z_700mb', 'biomass_12Z']]



#therest.to_csv('fire_features.csv')

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,TLML_12Z,QLML_12Z,SPEEDLML_12Z,PS_12Z,T_12Z_700mb,T_12Z_500mb,QV_12Z_700mb,fccs,slp,asp,biomass_12Z,geometry
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## Helper Functions

In [4]:
#this function calculates the area of each grid in a given grid cell
# LAT and LON should be 2d
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LAT[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    lon_corners = (LON[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LON[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    return lat_corners, lon_corners


In [5]:
# this function calculates the overlap between the given shape (fire_shape) and given grid (LAT, LON). 
#LAT and LON should be 2D

def calculate_overlaps(lat, lon, lat_corners, lon_corners, lat_start, lat_stop, lon_start, lon_stop, fire_shape):
    overlaps = np.zeros(lat.shape)
    # buffer the fire shape so it has a non zero area
    if fire_shape.area ==0:
        fire_shape = fire_shape.buffer(0.01)
    # find the intersection between the first day's polygon and the  grid
    for ii in range(lat_start, lat_stop):
        for jj in range(lon_start, lon_stop):
            #print('ctr', lat[ii,jj], lon[ii,jj])
            """ SW_lat = lat_corners[ii-1, jj-1]
            SW_lon = lon_corners[ii-1, jj-1]
            SE_lat = lat_corners[ii-1, jj-1]
            SE_lon = lon_corners[ii-1, jj]
            NW_lat = lat_corners[ii, jj-1]
            NW_lon = lon_corners[ii, jj-1]
            NE_lat = lat_corners[ii, jj]
            NE_lon = lon_corners[ii, jj]"""
            poly_cell = Polygon([(lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1])])
            overlap_cell = fire_shape.intersection(poly_cell)
            #overlaps are the fraction of the POLYGON in each grid cell
            overlaps[ii,jj] = overlap_cell.area/fire_shape.area
    
    return overlaps

In [32]:
# for each incident
    #for each day up until the second to last one
        #for each day in the forecast 
            # calculate emissions using the most recently available polygon
            
ndays=2            
# dictionaries that track the variables, dimensions, times and levels
varis = {'/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['biomass']}

#0=lat, 1=lon, 2=time, 3=level 
dims = {'/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4':['lat', 'lon', 'time']}

times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'biomass':12}
table_varis = ['biomass_12Z_today', 'biomass_12Z_tomorrow']
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'
ndays_forecast = 1

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                             'biomass_12Z_today': np.nan*np.zeros(num_days),'biomass_12Z_tomorrow': np.nan*np.zeros(num_days),
                             'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe

#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
print(len(unique_days))
for dd in range(len(unique_days)):
    
    """ #define the incident we are looking at
    inci = unique_incidents[dd]
    print(dd, inci)
    
    #get the subset of the fire_features relevant for that subset
    fire_features_sub = fire_features[fire_features['Incident Number']==inci]
    fire_features_sub = fire_features_sub.reset_index(drop=True) #reset the indices beecause we dropped some
    #print(fire_features_sub)
    ndays_incident = len(fire_features_sub)
    #print(ndays_incident)
    
    #loop over the days in the incident
    for ii in range(ndays_incident):
        curr_day = np.datetime64(fire_features_sub.loc[ii, 'Current Day'])
        print(curr_day) #get the current day
    """
    # loop over the number of days to forecast
    for jj in range(ndays):
        date=np.datetime64(unique_days[dd])+np.array(jj, 'timedelta64[D]')
        date=str(date)
        yr = date[0:4]
        mo = date[5:7]
        dy = date[8:10]
        print(yr, mo,dy)

        #load in the most recent polygon

        #load in the data
        for i in varis : #loop over the file names
            #get the path to the file
            path = '/data2/lthapa/'+yr+i
            print(path)
            if '*' in path:
                path = path.replace('*', yr+mo+dy)
            print(path)
            dat = nc.Dataset(path)
            ndims = len(dims[i])
            lat = dat[dims[i][0]][:]
            lon = dat[dims[i][1]][:]
            t = dat[dims[i][2]][:]
            if len(lat.shape)==1:
                lon, lat = np.meshgrid(lon, lat)

            #get the corners of the lat/lon grid
            # NEW WAY WITH CURRENT DAY'S POLYGON
            lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
            inds = np.where(fire_features['Current Day']==unique_days[dd])[0]
            polys = fire_features['geometry'].iloc[inds].values
            print(inds)
            for p in range(len(inds)):
                """OLD WAY WITH THE MOST RECENT POLYGON
                #what is the most recent day that exists in the fire_features_sub??
                poly_ind = np.where(fire_features_sub['Current Day'].values==date)
                if len(poly_ind[0])==0: #one day fires and last day
                    poly_ind=ii
                    p = fire_features_sub.loc[poly_ind, 'geometry']
                else:
                    p = fire_features_sub.loc[poly_ind, 'geometry'].values[0]"""
                #print(p)
                poly = polys[p]
                #add a halo to the polygon
                p_buff = poly.buffer(1)
                # get the bounds of the buffered polygon
                w,s,e,n = p_buff.bounds
                #print(w,s,e,n)
                s_ind = np.unique(np.argmin(np.abs(lat-s), axis=0))[0]
                n_ind = np.unique(np.argmin(np.abs(lat-n), axis=0))[0]
                w_ind = np.unique(np.argmin(np.abs(lon-w), axis=1))[0]
                e_ind = np.unique(np.argmin(np.abs(lon-e), axis=1))[0]

                #do the overlaps
                overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners,s_ind,n_ind,w_ind,e_ind, poly) #calculate the overlap between the poly and the lat/lon grid
                #print(np.where(overlaps!=0))
                overlaps[overlaps!=0] = 1
                for j in varis[i]: #loop over the variable names, which are lists
                    var = dat[j][:]
                    ind_time = np.argmin(np.abs(times[j]*60-t))
                    #filter the variable by time
                    var = var[ind_time, :, :]
                    #print(var.shape)
                    var_weighted = np.multiply(var,overlaps)
                    var_ave = np.nansum(var_weighted) # because overlaps is a mask of 1s and 0s, this is the sum of the emissions rates which fall into the polygon
                    name_in_table = table_varis[jj]
                    #ind_fill = np.where((fire_features['Current Day']==str(date)) &(fire_features['Incident Number']==inci))[0]
                    #print(ind_fill[0])
                    #print(name_in_table)
                    fire_features.loc[inds[p], name_in_table] = var_ave
            
            
        
fire_features

442
2019 02 17
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190217.nc4
[470]
2019 02 18
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190218.nc4
[470]
2019 02 18
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190218.nc4
[471]
2019 02 19
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190219.nc4
[471]
2019 02 24
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190224.nc4
[222]
2019 02 25
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190225.nc4
[222]
2019 03 06
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190306.nc4
[1025]
2019 03 07
/data2/lthapa/2019

2019 05 26
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190526.nc4
[37]
2019 05 27
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190527.nc4
[37]
2019 05 27
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190527.nc4
[38]
2019 05 28
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190528.nc4
[38]
2019 05 28
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190528.nc4
[39]
2019 05 29
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190529.nc4
[39]
2019 05 29
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190529.nc4
[  40 1426]
2019 05 30
/data2/lthapa/2019/QFED

2019 06 21
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190621.nc4
[  12  174  497 1198 1409 1529 1556 2266 2278]
2019 06 21
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190621.nc4
[  13  175  498 1410 1530 2167 2267 2279]
2019 06 22
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190622.nc4
[  13  175  498 1410 1530 2167 2267 2279]
2019 06 22
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190622.nc4
[  14  176  499 1411 1531 2168 2268 2280]
2019 06 23
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190623.nc4
[  14  176  499 1411 1531 2168 2268 2280]
2019 06 23
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.201906

2019 07 13
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190713.nc4
[ 116  191  194  686 1341 1659 1980 2255 2415 2454 2479]
2019 07 14
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190714.nc4
[ 116  191  194  686 1341 1659 1980 2255 2415 2454 2479]
2019 07 14
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190714.nc4
[  95  102  117  126  195  733 1342 1981 2243 2256 2416 2455 2480]
2019 07 15
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190715.nc4
[  95  102  117  126  195  733 1342 1981 2243 2256 2416 2455 2480]
2019 07 15
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190715.nc4
[  96  107  114  118  127  196 1191 1343 1562 1982 2244 2257 2429 2456
 2481]
2019 07 16
/da

2019 08 01
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190801.nc4
[ 142  362  433  474  590  751 1088 1237 1275 1359 1397 1584 1671 1861
 1998 2039 2150 2152 2261 2330 2347 2359 2426]
2019 08 01
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190801.nc4
[ 143  363  434  475  591  752 1089 1238 1276 1360 1585 1672 1758 1862
 1999 2040 2151 2153 2331 2348 2360 2427]
2019 08 02
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190802.nc4
[ 143  363  434  475  591  752 1089 1238 1276 1360 1585 1672 1758 1862
 1999 2040 2151 2153 2331 2348 2360 2427]
2019 08 02
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2019/QFED/WESTUS_qfed2.emis_pm25.006.20190802.nc4
[ 144  364  435  476  592  641  644  753  796 1090 1239 1277 1338 1361
 1586 1664 1673 1759 1863 2000 2041 2068 2101 2154 2332 2

...

2020 10 01
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201001.nc4
[3264 3641 3832 4115 4226 4484 4534 4624 4691 4762 4871 5127 5231 5357
 5492 5546 5688 5754 5797 5940 6028 6203 6316 6401 6486 6698 6728 6823
 6864 6916 7014 7108 7158 7190 7224 7286 7317 7397 7444 7545 7614 7677
 7715 7747 7808 7889 7918 7954 7971 7984 8016 8071]
2020 10 02
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201002.nc4
[3264 3641 3832 4115 4226 4484 4534 4624 4691 4762 4871 5127 5231 5357
 5492 5546 5688 5754 5797 5940 6028 6203 6316 6401 6486 6698 6728 6823
 6864 6916 7014 7108 7158 7190 7224 7286 7317 7397 7444 7545 7614 7677
 7715 7747 7808 7889 7918 7954 7971 7984 8016 8071]
2020 10 02
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201002.nc4
[3265 3642 3833 4116 4227 4485 4535 4625 4692 4763 4872 5128 5358 5493
 55

2020 10 12
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201012.nc4
[3275 3652 3843 4126 4237 4545 4702 5138 5368 5557 5699 5808 5951 6927
 7025 7119 7235 7328 7408 7455 7556 7625 7836 7850 7900 7929 7964 8027]
2020 10 13
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201013.nc4
[3275 3652 3843 4126 4237 4545 4702 5138 5368 5557 5699 5808 5951 6927
 7025 7119 7235 7328 7408 7455 7556 7625 7836 7850 7900 7929 7964 8027]
2020 10 13
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201013.nc4
[3276 3653 3844 4127 4238 4546 4703 5139 5369 5558 5700 5809 5952 6928
 7026 7120 7236 7329 7409 7456 7557 7626 7837 7851 7901 7930 8028]
2020 10 14
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201014.nc4
[3276 3653 3844 4127 4238 4546 4703 5139 5369 55

2020 10 27
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201027.nc4
[3290 3667 3858 4141 4252 4560 4717 5383 5572 5823 5966 6942 7134 7250
 7343 7470 7571 7915 7944 8129 8138 8139 8150]
2020 10 28
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201028.nc4
[3290 3667 3858 4141 4252 4560 4717 5383 5572 5823 5966 6942 7134 7250
 7343 7470 7571 7915 7944 8129 8138 8139 8150]
2020 10 28
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201028.nc4
[3291 3668 3859 4142 4253 4561 4718 5384 5573 5824 5967 6943 7251 7344
 7471 7572 7945 8130 8140 8151]
2020 10 29
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201029.nc4
[3291 3668 3859 4142 4253 4561 4718 5384 5573 5824 5967 6943 7251 7344
 7471 7572 7945 8130 8140 8151]
2020 10 29
/data2/lthapa/2020/

2020 11 17
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201117.nc4
[4273 4581 6963 7364 7491 8125 8146 8147]
2020 11 18
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201118.nc4
[4273 4581 6963 7364 7491 8125 8146 8147]
2020 11 18
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201118.nc4
[4274 6964 7365 7492]
2020 11 19
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201119.nc4
[4274 6964 7365 7492]
2020 11 19
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201119.nc4
[4275 6965 7366 8143]
2020 11 20
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.*.nc4
/data2/lthapa/2020/QFED/WESTUS_qfed2.emis_pm25.006.20201120.nc4
[4275 6965 7366 8143]
2020 11 20
/data2/lthapa/2020/QFED/W

,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points,biomass_12Z_today,biomass_12Z_tomorrow,geometry
0,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0,7.627477e-10,9.852367e-13,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,10663171.0,LONE MOUNTAIN,2019-06-09,33.808056,-105.738611,2.0,0.000000e+00,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,10663171.0,LONE MOUNTAIN,2019-06-10,33.808056,-105.738611,0.0,0.000000e+00,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,10663171.0,LONE MOUNTAIN,2019-06-11,33.808056,-105.738611,0.0,0.000000e+00,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,10663171.0,LONE MOUNTAIN,2019-06-12,33.808056,-105.738611,0.0,0.000000e+00,0.000000e+00,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...,...,...,...,...,...,...,...,...
8156,11979237.0,Trail,2020-11-02,34.147222,-112.117778,2.0,0.000000e+00,0.000000e+00,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,11980186.0,SANDERSON,2020-12-13,33.889444,-117.070833,11.0,1.553367e-09,1.298944e-10,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,11980587.0,CREEK 5,2020-12-24,33.402646,-117.266795,2.0,0.000000e+00,7.784754e-10,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0,7.784754e-10,0.000000e+00,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


In [33]:
therest = fire_features[['Incident Number', 'Fire Name', 'Current Day','Lat Fire', 'Lon Fire','Number of VIIRS points', 'biomass_12Z_today', 'biomass_12Z_tomorrow' ]]
therest

#therest.to_csv('fire_features_precip_2019.csv', index=False)
#therest.to_csv('fire_features_precip_2020.csv', index=False)
therest.to_csv('fire_labels_1dayout_day0poly.csv', index=False)

In [45]:
np.datetime64(fire_features_sub.loc[len(fire_features_sub)-1, 'Current Day'])

numpy.datetime64('2019-07-02')

In [12]:
print(curr_day)

2019-07-22


In [15]:
print(np.where(fire_features['Current Day']==curr_day))

(array([], dtype=int64),)
